## 丢弃法

除了前一节介绍的权重衰减以外，深度学习模型常常使用丢弃法$（dropout）$ 来应对过拟合问题。丢弃法有一些不同的变体。本节中提到的丢弃法特指倒置丢弃法$（inverted dropout）$。

在3.8节中描述的多层感知机的计算表达式如下：
<center>
    $ h_i= \phi(x_1w_{1i}+x_2w_{2i}+x_3w_{3i}+x_4w_{4i}+b_i)$
</center>
当对该隐藏层使用丢弃法的时，该层的隐藏单元将有一定概率被丢弃掉。设丢弃概率为$p$，那么有$p$的概率$h_i$会被清零，有$1-p$的概率$h_i$会除以$1-p$做拉伸。丢弃概率是丢弃法的超参数。具体来说，设随机变量${\xi}_i$为$0$和$1$的概率分别为$p$和$1-p$。使用丢弃法时我们计算新的隐藏单元$h_{inew}$
<center>
    $ h_{inew} =  \frac{\xi}{1-p}h_i$
</center>
由于$E({\xi}_i)=1-p$，因此
<center>
    $ E(h_{inew})=\frac{E({\xi}_i)}{1-p}h_i=h_i $
</center>
即**丢弃法不改变其输入的期望值**。

由于在训练中隐藏层神经元的丢弃是随机的，即$h_1,…,h_5$ 都有可能被清零，输出层的计算无法过度依赖$h_1,…,h_5$中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。

在**测试**模型时，我们为了拿到更加确定性的结果，一般不使用丢弃法。

**从零开始实现**

In [1]:
%matplotlib inline
import torch
import numpy as np
import torch.nn as nn
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

def dropout(X, drop_prob):
    X=X.float()
    assert 0 <= drop_prob <=1
    keep_prob = 1-drop_prob
    
    if keep_prob == 0:
        return torch.zeros_like(X)
    mask = (torch.rand(X.shape)<keep_prob).float()
    
    return mask*X/keep_prob

In [2]:
X = torch.arange(16).view(2,8)
dropout(X, 0)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [3]:
dropout(X, 0.5)

tensor([[ 0.,  0.,  4.,  0.,  0., 10.,  0., 14.],
        [16.,  0., 20.,  0., 24.,  0., 28., 30.]])

In [4]:
dropout(X, 1.0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])